In [1]:
import pandas as pd

df_manuf_brecha = pd.read_csv('../results/bea/majority_owned_nonbank/brecha_productividad_all_sectors.csv',
                              sep=" ")
df_manuf_brecha = df_manuf_brecha[['country', 'sector', 'year', 
                                     'ipt_arg_99_index',  'ipt_bench_99_index', 
                                     'brecha_anio_base' ]]
df = df_manuf_brecha.sort_values(by=['country', 'sector', 'year' ])


df.sample(4)

,country,sector,year,ipt_arg_99_index,ipt_bench_99_index,brecha_anio_base
1774,Denmark,Computers and electronic products,2009,0.724559,NaN,NaN
2194,Greece,Professional services,2011,0.477095,2.473586,NaN
1085,Poland,Electrical equipment,2005,-0.488319,1.873156,NaN
2941,Norway,Manufacturing without Transportation Equipment,2015,1.141753,-1.420081,NaN


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3842 entries, 0 to 3641
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country             3842 non-null   object 
 1   sector              3842 non-null   object 
 2   year                3842 non-null   int64  
 3   ipt_arg_99_index    3573 non-null   float64
 4   ipt_bench_99_index  3195 non-null   float64
 5   brecha_anio_base    164 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 210.1+ KB


In [3]:
# Función para calcular la brecha
def calcular_brecha(row):
    if pd.notna(row['brecha_anio_base']):
        return row['brecha_anio_base']
    else:
        filtro_anterior = (df['year'] == row['year'] - 1) & (df['country'] == row['country']) & (df['sector'] == row['sector'])
        if df[filtro_anterior].shape[0] > 0:
            brecha_anio_anterior = df.loc[filtro_anterior, 'brecha_calculada'].values[0]
            ipt_arg_ratio = row['ipt_arg_99_index'] / df.loc[filtro_anterior, 'ipt_arg_99_index'].values[0]
            ipt_bench_ratio = row['ipt_bench_99_index'] / df.loc[filtro_anterior, 'ipt_bench_99_index'].values[0]
            return brecha_anio_anterior * (ipt_arg_ratio / ipt_bench_ratio)
        else:
            return None


In [4]:

# # Iterar sobre las filas y calcular las brechas
for index, row in df.iterrows():
    df.at[index, 'brecha_calculada'] = calcular_brecha(row)

df.head()

,country,sector,year,ipt_arg_99_index,ipt_bench_99_index,brecha_anio_base,brecha_calculada
0,Denmark,All Industries Total,1999,1.000000,1.000000,0.435054,0.435054
167,Denmark,All Industries Total,2000,0.965080,0.903858,NaN,0.464522
336,Denmark,All Industries Total,2001,0.943760,0.804006,NaN,0.510676
510,Denmark,All Industries Total,2002,1.257086,0.788811,NaN,0.693323
687,Denmark,All Industries Total,2003,1.378260,0.885420,NaN,0.677212


In [5]:
# Agrupar por país y aplicar el cálculo
resultados_por_pais = pd.DataFrame()

for country, group_df in df.groupby(['country', 'sector']):
    group_df['brecha_calculada'] = group_df.apply(calcular_brecha, axis=1)
    resultados_por_pais = pd.concat([resultados_por_pais, group_df])

# Mostrar el DataFrame resultante
resultados_por_pais.head()

,country,sector,year,ipt_arg_99_index,ipt_bench_99_index,brecha_anio_base,brecha_calculada
0,Denmark,All Industries Total,1999,1.000000,1.000000,0.435054,0.435054
167,Denmark,All Industries Total,2000,0.965080,0.903858,NaN,0.464522
336,Denmark,All Industries Total,2001,0.943760,0.804006,NaN,0.510676
510,Denmark,All Industries Total,2002,1.257086,0.788811,NaN,0.693323
687,Denmark,All Industries Total,2003,1.378260,0.885420,NaN,0.677212


In [6]:
resultados_por_pais.to_csv('../results/bea/majority_owned_nonbank/productivity_gap_all_sectors_sep.csv',
          index   = False,
          sep=';', decimal=',')
resultados_por_pais.to_csv('../results/bea/majority_owned_nonbank/productivity_gap_all_sectors.csv',
          index   = False)

In [9]:
# resultados_por_pais[resultados_por_pais['country']=='Germany'].head()
resultados_por_pais[(resultados_por_pais['country']=='Germany') & 
                    (resultados_por_pais['sector']== 'Manufacturing without Transportation Equipment')].head()
# resultados_por_pais[resultados_por_pais['sector']=='Transportation Equipment'].head()


,country,sector,year,ipt_arg_99_index,ipt_bench_99_index,brecha_anio_base,brecha_calculada
157,Germany,Manufacturing without Transportation Equipment,1999,1.000000,NaN,NaN,NaN
326,Germany,Manufacturing without Transportation Equipment,2000,0.892392,NaN,NaN,NaN
500,Germany,Manufacturing without Transportation Equipment,2001,0.927039,NaN,NaN,NaN
677,Germany,Manufacturing without Transportation Equipment,2002,0.824537,NaN,NaN,NaN
848,Germany,Manufacturing without Transportation Equipment,2003,1.317542,NaN,NaN,NaN
